<img src="image/arch.png"/>

#  Getting Started with Snowpark

In this notebook, we are going to use the [Snowpark](https://docs.snowflake.com/en/developer-guide/snowpark/index.html) Scala API. We will use a DEMO database with schema TELCO to demonstrate some new features of snowflake.

## Configure the Jupyter Notebook

In this step, we will write some boiler palte code to include important Maven repository for the required Snowpark libraries, configure the compiler for the Scala REPL, and set-up a directory for classes generated by the Scala REPL.

In [22]:
import sys.process._

val osgeoRepo = coursierapi.MavenRepository.of("https://repo.osgeo.org/repository/release")
interp.repositories() ++= Seq(osgeoRepo)

import sys.process._
val replClassPath = "/home/jovyan/work/repl"
s"mkdir -p $replClassPath" !

interp.configureCompiler(_.settings.outputDirs.setSingleOutput(replClassPath))
interp.configureCompiler(_.settings.Yreplclassbased)
interp.load.cp(os.Path(replClassPath))

import sys.process._


osgeoRepo: coursierapi.MavenRepository = MavenRepository(https://repo.osgeo.org/repository/release)
import sys.process._

replClassPath: String = "/home/jovyan/work/repl"
res21_5: Int = 0

## Import Snowpark Libraries and Open a session

In this step, snowpark libraries are imported and a new session is opened. In order to open a session, we will add the parameters to a text file called telco_connection.txt. 

Navigate to **/home/jovyan/work**, there is a file called telco_connection.txt that can be customized to your Snowflake Account Name, your credentials, and the virtual warehouse name you want to use for this tutorial. 
We will demonstrate Snowpark's ability to perform ELT operations

In [32]:
// Import the Snowpark library from Maven.
import $ivy.`com.snowflake:snowpark:0.8.0`

import com.snowflake.snowpark._
import com.snowflake.snowpark.types._
import com.snowflake.snowpark.functions._

val session = Session.builder.configFile("/home/jovyan/telecom/telco_connection.txt").create

// Add the directory for REPL classes that you created earlier.
session.addDependency(replClassPath)

// Add the classes for Ammonite as dependencies.
// Required for UDF creation
def addClass(session: Session, className: String): String = {
  var cls1 = Class.forName(className)
  val resourceName = "/" + cls1.getName().replace(".", "/") + ".class"
  val url = cls1.getResource(resourceName)
  val path = url.getPath().split(":").last.split("!").head
  session.addDependency(path)
  path
}

addClass(session, "pprint.TPrintColors")

import $ivy.$                             


import com.snowflake.snowpark._

import com.snowflake.snowpark.types._

import com.snowflake.snowpark.functions._


session: Session = com.snowflake.snowpark.Session@142b824a
defined function addClass
res31_7: String = "/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/pprint_2.12/0.5.9/pprint_2.12-0.5.9.jar"

In [33]:
// Parameters for run:
val trainDb="DEMO"
val trainSchema="TELCO"

trainDb: String = "DEMO"
trainSchema: String = "TELCO"

In [34]:
val rawData = session.table("RAW_PARQUET")
//rawData.show(5)
val setContext = "USE SCHEMA " + trainDb + "." + trainSchema + " ;"
session.sql(setContext).collect()

rawData: Updatable = com.snowflake.snowpark.Updatable@7dae8774
setContext: String = "USE SCHEMA DEMO.TELCO ;"
res33_2: Array[Row] = Array(Row[Statement executed successfully.])

# STEP 5 - Snowpark Transformations

The Snowpark API provides programming language constructs for building SQL statements. It's a new developer experience which enables us to build code in :-

<b><li>  Language of our choice </li></b>
<b><li> Tool of our choice and </li></b>
<b><li> Lazy execution to prevent multiple network hops to server </li></b>

Once the customer data is available in the RAW schema, we can use snowpark to create dimensions and fact tables. We will use the RAW_PARQUET table to create following tables -
    
<li> DEMOGRAPHICS </li>
<li> LOCATION </li>
<li> STATUS </li>
<li> SERVICES </li>

We will also transform and clean the data using Snowpark dataframe API


In [27]:
val setContext = "USE SCHEMA " + trainDb + "." + trainSchema
session.sql(setContext).collect()

val dfDemographics = session.table("RAW_PARQUET")
   .select(col("CUSTOMERID"),
           replace(col("COUNT"),lit("NULL"),lit(1)).name("COUNT"),
          replace(col("GENDER"),lit("NULL"),lit("Male")).name("GENDER"),
           replace(col("SENIOR CITIZEN"),lit("NULL"),lit('N')).name("SENIORCITIZEN"),
           replace(col("PARTNER"),lit("NULL"),lit('N')).name("PARTNER"),
           replace(col("DEPENDENTS"),lit("NULL"),lit('N')).name("DEPENDENTS")          
          )

//dfRaw.schema
//dfRaw.show


dfDemographics.write
   .mode(SaveMode.Append)
   .saveAsTable("DEMOGRAPHICS")

setContext: String = "USE SCHEMA DEMO.TELCO"
res26_1: Array[Row] = Array(Row[Statement executed successfully.])
dfDemographics: DataFrame = com.snowflake.snowpark.DataFrame@726d3374

In [28]:
val setContext = "USE SCHEMA " + trainDb + "." + trainSchema
session.sql(setContext).collect()

val dfLocation = session.table("RAW_PARQUET")
   .select(col("CUSTOMERID"),
           replace(col("COUNTRY"),lit("NULL"),lit("None")).name("COUNTRY"),
           replace(col("STATE"),lit("NULL"),lit("None")).name("STATE"),
           replace(col("CITY"),lit("NULL"),lit("None")).name("CITY"),
           replace(col("ZIP CODE"),lit("NULL"),lit(0)).name("ZIPCODE"),
           replace(col("LAT LONG"),lit("NULL"),lit("0.0,0.0")).name("LATLONG"),
           replace(col("LATITUDE"),lit("NULL"),lit("0.0")).name("LATITUDE"),
           replace(col("LONGITUDE"),lit("NULL"),lit("0.0")).name("LONGITUDE")          
          )


dfLocation.write
   .mode(SaveMode.Append)
   .saveAsTable("LOCATION")

setContext: String = "USE SCHEMA DEMO.TELCO"
res27_1: Array[Row] = Array(Row[Statement executed successfully.])
dfLocation: DataFrame = com.snowflake.snowpark.DataFrame@528dc432

In [29]:
val setContext = "USE SCHEMA " + trainDb + "." + trainSchema
session.sql(setContext).collect()

val dfServices = session.table("RAW_PARQUET")
           .select(col("CUSTOMERID"),
           col("TENURE MONTHS").as("TENUREMONTHS"),
           replace(col("PHONE SERVICE"),lit("NULL"),lit('N')).name("PHONESERVICE"),
           replace(col("MULTIPLE LINES"),lit("NULL"),lit("No")).name("MULTIPLELINES"),
           replace(col("INTERNET SERVICE"),lit("NULL"),lit("No")).name("INTERNETSERVICE"),
           replace(col("ONLINE SECURITY"),lit("NULL"),lit("No")).name("ONLINESECURITY"),
           replace(col("ONLINE BACKUP"),lit("NULL"),lit("No")).name("ONLINEBACKUP"),
           replace(col("DEVICE PROTECTION"),lit("NULL"),lit("No")).name("DEVICEPROTECTION"),
           replace(col("TECH SUPPORT"),lit("NULL"),lit('N')).name("TECHSUPPORT"),
           replace(col("STREAMING TV"),lit("NULL"),lit("No")).name("STREAMINGTV"),
           replace(col("STREAMING MOVIES"),lit("NULL"),lit("No")).name("STREAMINGMOVIES"),
           replace(col("CONTRACT"),lit("NULL"),lit("Month-to-month")).name("CONTRACT"),
           replace(col("PAPERLESS BILLING"),lit("NULL"),lit('Y')).name("PAPERLESSBILLING"),
           replace(col("PAYMENT METHOD"),lit("NULL"),lit("Mailed check")).name("PAYMENTMETHOD"),
           replace(col("MONTHLY CHARGES"),lit(" "),lit(20)).name("MONTHLYCHARGES"),
           replace(col("TOTAL CHARGES"),lit(" "),lit(0)).name("TOTALCHARGES")
              
          )


dfServices.write
   .mode(SaveMode.Append)
   .saveAsTable("SERVICES")

setContext: String = "USE SCHEMA DEMO.TELCO"
res28_1: Array[Row] = Array(Row[Statement executed successfully.])
dfServices: DataFrame = com.snowflake.snowpark.DataFrame@59860e89

In [30]:
val trainDb="DEMO"
val trainSchema="TELCO"

val setContext = "USE SCHEMA " + trainDb + "." + trainSchema
session.sql(setContext).collect()


val dfStatus = session.table("RAW_PARQUET")
   .select(col("CUSTOMERID"),
           replace(col("CHURN LABEL"),lit("NULL"),lit('N')).name("CHURNLABEL"),
          replace(col("CHURN VALUE"),lit("NULL"),lit(0)).name("CHURNVALUE"),
           replace(col("CHURN SCORE"),lit("NULL"),lit(50)).name("CHURNSCORE"),
           replace(col("CLTV"),lit("NULL"),lit(0)).name("CLTV"),
           replace(col("CHURN REASON"),lit("NULL"),lit("do not know")).name("CHURNREASON")          
          )

//dfRaw.schema
//dfRaw.show

dfStatus.write
   .mode(SaveMode.Append)
   .saveAsTable("STATUS")

trainDb: String = "DEMO"
trainSchema: String = "TELCO"
setContext: String = "USE SCHEMA DEMO.TELCO"
res29_3: Array[Row] = Array(Row[Statement executed successfully.])
dfStatus: DataFrame = com.snowflake.snowpark.DataFrame@365245cc

# Lets look at an example query

This shows one of many uses of snowpark. You can run dataframes lazily.

In [31]:
val dfLoc = session.table("LOCATION")
val dfServ = session.table("SERVICES")

val dfJoin = dfLoc.join(dfServ,dfLoc.col("CUSTOMERID") === dfServ.col("CUSTOMERID"))

val dfResult = dfJoin.select(
                            col("CITY"),col("CONTRACT"),col("TOTALCHARGES"))
                            .groupBy(col("CITY"),col("CONTRACT"))
                            .sum(col("TOTALCHARGES"))

dfResult.show()

-----------------------------------------------------------------
|"CITY"                  |"CONTRACT"      |"SUM(TOTALCHARGES)"  |
-----------------------------------------------------------------
|San Dimas               |Month-to-month  |2379459.2            |
|Glendale                |Month-to-month  |7367728.8            |
|Rosemead                |Month-to-month  |3494255.2            |
|Lynwood                 |Month-to-month  |308924.8             |
|Stevenson Ranch         |Month-to-month  |550679.2000000001    |
|Wrightwood              |Month-to-month  |1278892.0            |
|March Air Reserve Base  |Month-to-month  |458735.2             |
|Hemet                   |Month-to-month  |911392.8             |
|Lindsay                 |Month-to-month  |1533092.0            |
|Lancaster               |Month-to-month  |861628.7999999999    |
-----------------------------------------------------------------



dfLoc: Updatable = com.snowflake.snowpark.Updatable@7317f3a
dfServ: Updatable = com.snowflake.snowpark.Updatable@455a2927
dfJoin: DataFrame = com.snowflake.snowpark.DataFrame@7fae136f
dfResult: DataFrame = com.snowflake.snowpark.DataFrame@67b49b40

# STEP 6: Building features using serverless tasks

Snowflake now supports Serverless tasks for scheduling and chaining multiple tasks. Advantage for using this option is that you don't have to define a predefined warehouse to execute the statements. Compute utilization can ramp up and down based on the workload.

We can use this feature to build features and store it our feature store. This way Data science team use the final output of this end to end pipeline to find the Customer churn